In [9]:
import numpy as np
import random

In [10]:
def communication_term(K, W, H, rho_c=1.0, sigma_n2=1e-3):
  ans = 0.0
  for k in range(K):
    min_R_kj = float("inf")
    for j in range(k, K):
      h_j_H = H[:, j].conj().T
      w_k = W[:, k]

      num = np.abs((h_j_H @ w_k)) ** 2
      den = np.sum([np.abs((h_j_H @ W[:, i]))**2 for i in range(k+1, K)]).item() + sigma_n2
      val = np.log2(1 + num/den)

      if val < min_R_kj:
        min_R_kj = val
    ans += min_R_kj # 0.0 + R1 + R2 +...+ RK

  return rho_c * ans

In [11]:
def sensing_term(N, K, M, W, thetas, _lambda, d, rho_r=1.0):
  ans = 0.0
  A = np.zeros((N, M), dtype=complex) # each col is a(theta_m)
  for i in range(N): # row
    for m in range(M): # col
      theta_m = thetas[m]
      A[i, m] = np.exp(1j * 2*np.pi/_lambda * d * i * np.sin(theta_m))

  # iterating through cols of W or rows of W.T
  R_w = np.sum([np.outer(w_i, w_i.conj()) for w_i in W.T], axis=0) # shape = (N, N)
  for m in range(M): # iterating through cols
    col_m = A[:, m]
    ans += np.abs(col_m.conj().T @ R_w @ col_m)

  return rho_r * ans

In [12]:
def f(N, K, M, W, H, thetas, _lambda, d, rho_c, rho_r, sigma_n2):
  return communication_term(K, W, H, rho_c, sigma_n2) + sensing_term(N, K, M, W, thetas, _lambda, d, rho_r)

In [13]:
# Test function to call f() with initialized values
def test_f():
    """Initialize test parameters and call f(.)"""
    N = 4  # Number of antennas
    K = 3  # Number of beamformers
    M = 2  # Number of target directions
    _lambda = 1.0  # Wavelength (assumed to be 1 for simplicity)
    d = 0.5  # Antenna spacing (half wavelength spacing)
    rho_c = 1.0  # Communication-related parameter
    rho_r = 1.0  # Sensing-related parameter
    sigma_n2 = 1e-3  # Noise power

    # Random values for W and H (beamforming vectors and channel matrix)
    W = np.random.rand(N, K) + 1j * np.random.rand(N, K)  # Complex beamforming vectors
    H = np.random.rand(N, K) + 1j * np.random.rand(N, K)  # Complex channel matrix

    # Target directions (angles in radians)
    thetas = np.array([0.2, 0.5])  # Two target directions for testing

    # Call function f() with the initialized parameters
    result = f(N, K, M, W, H, thetas, _lambda, d, rho_c, rho_r, sigma_n2)

    print("Result from f():", result)

test_f()

Result from f(): 36.282684404242765
